<a href="https://colab.research.google.com/github/mhpbreugem/BBP/blob/main/bbp_2s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
using Distributions, Random, NLsolve, LinearAlgebra, Printf, Optim, BenchmarkTools, KrylovKit

2 stock code with symmetrixal distributions and zero correlation between stock payoff

In [3]:
# Chapter 0: Parameters

# Economic parameters
const β = 0.95
const γ = 4.0
const e0 = 5.0
const EΠ = 1.0
const Π0 = 1.0
const θXbar = 1.0
const θYbar = 1.0
const μθX = 0.5
const σθX = 0.2
const μθY = 0.5
const σθY = 0.1
const σΠ = 0.4
const W0 = e0

const ΞЮ = [repeat([x], 1) for x in range(0.1, stop=1.0, length=10)]

# Grid parameters
const Nσπ = 3
const NNσπ = 1
const Nπ = 2 * Nσπ * NNσπ + 1  # number of grid points for payoff

const NσΠ = 2
const NNσΠ = 1
const NΠ = 2 * NσΠ * NNσΠ + 1  # number of grid points for payoff

const NθX = 3
const NθY = 3
const NσθX = 1
const NσθY = 1

# Derived midpoints
const μπi = (Nπ + 1) ÷ 2
const μΠi = (NΠ + 1) ÷ 2
const μθXi = (NθX + 1) ÷ 2

# Total number of states
const NS = Nπ
const NN = NθX *NθX * NθY * NΠ* NΠ

675

In [4]:
# Chapter 1: Grid Construction

# Conjectured Payoffs
const ππ = collect(range(EΠ - Nσπ*σΠ, EΠ + Nσπ*σΠ, length=Nπ))
const πω = pdf.(Normal(EΠ, σΠ), ππ)
const πΩ = πω ./ sum(πω)

# Payoff Parameters
const Δ1 = Π0
const Δ2 = ππ

# True Payoffs
const ΞΠ = EΠ .+ σΠ .* range(-NσΠ, NσΠ, length=NΠ)
const ππ_to_index = Dict(round(ππ[πi], digits=8) => πi for πi in 1:Nπ)
const ΞΠi = [ππ_to_index[round(ΞΠ[Πi], digits=8)] for Πi in 1:NΠ]

# Signals
const S = ππ
const ϵ = S .- ππ'

@inline function ψϵω(Ю, ϵx)
    σϵ = 1 / sqrt(Ю)
    pdf(Normal(0.0, σϵ), ϵx)
end

function φSΩπ(Ю)
    σϵ = 1 / sqrt(Ю[1])
    SΩ = [pdf(Normal(π, σϵ), s) for π in ππ, s in ππ]
    SΩnorm = SΩ ./ sum(SΩ, dims=2)
    return SΩnorm
end

# Noise Trader Demand
const ΞθX = μθX .+ σθX .* range(-NσθX, NσθX, length=NθX)
const ΞθY = μθY .+ σθY .* range(-NσθY, NσθY, length=NθY)

const Σθ = [σθX^2 0 0; 0 σθX^2 0; 0 0 σθY^2]
const dist = MvNormal([μθX, μθX, μθY], Σθ)
θXYωω(xA, xB, y) = pdf(dist, [xA, xB, y])

θXYωω (generic function with 1 method)

In [5]:
θXYωω(0,0,0)

1.1419541587249434e-7

In [14]:
# Chapter 2: Learning

# Full learning (private + public information)
function φΞΩI(Ю, XAΣ, XBΣ, YΣ)
    ψϵ = ψϵω.(Ю, ϵ)
    θXY = θXYωω.(θXbar .- XAΣ, θXbar .- XBΣ, θYbar .- YΣ)
    ΞΩI = ψϵ * ((πΩ * πΩ') .* θXY) * ψϵ'
    ΞΩI ./= sum(ΞΩI)
return ΞΩI
end

###WARNING INCORRECT DIMENSIONALITY.. careful about summing over SAi x SBi

@btime φΞΩI([1], fill(1,(NS,NS)), fill(1,(NS,NS)), fill(1,(NS,NS)))

  8.613 μs (206 allocations: 16.25 KiB)


7×7 Matrix{Float64}:
 0.00969255  0.0136849  0.0168313  …  0.0168313  0.0136849  0.00969255
 0.0136849   0.0193216  0.023764      0.023764   0.0193216  0.0136849
 0.0168313   0.023764   0.0292279     0.0292279  0.023764   0.0168313
 0.0180333   0.0254612  0.0313152     0.0313152  0.0254612  0.0180333
 0.0168313   0.023764   0.0292279     0.0292279  0.023764   0.0168313
 0.0136849   0.0193216  0.023764   …  0.023764   0.0193216  0.0136849
 0.00969255  0.0136849  0.0168313     0.0168313  0.0136849  0.00969255

In [17]:
# Chapter 2: Learning

# Full learning (private + public information)
function φΞΩI(Ю, XAΣ, XBΣ, YΣ)
    ψϵ = ψϵω.(Ю, ϵ)
    θXY = θXYωω.(θXbar .- XAΣ, θXbar .- XBΣ, θYbar .- YΣ)
    ΞΩI = ψϵ * ((πΩ * transpose(πΩ)) .* θXY) * transpose(ψϵ)
    sum_ΞΩI = sum(ΞΩI)
    ΞΩI .= ΞΩI ./ sum_ΞΩI
    return ΞΩI
end

###WARNING INCORRECT DIMENSIONALITY.. careful about summing over SAi x SBi

@btime φΞΩI([1], fill(1,(NS,NS)), fill(1,(NS,NS)), fill(1,(NS,NS)))

  8.556 μs (206 allocations: 16.25 KiB)


7×7 Matrix{Float64}:
 0.00969255  0.0136849  0.0168313  …  0.0168313  0.0136849  0.00969255
 0.0136849   0.0193216  0.023764      0.023764   0.0193216  0.0136849
 0.0168313   0.023764   0.0292279     0.0292279  0.023764   0.0168313
 0.0180333   0.0254612  0.0313152     0.0313152  0.0254612  0.0180333
 0.0168313   0.023764   0.0292279     0.0292279  0.023764   0.0168313
 0.0136849   0.0193216  0.023764   …  0.023764   0.0193216  0.0136849
 0.00969255  0.0136849  0.0168313     0.0168313  0.0136849  0.00969255

In [ ]:
ψϵω.([1], ϵ)*Diagonal(πΩ)

7×7 Matrix{Float64}:
 0.00176853   0.0198886   0.0701159  …  0.0268469  0.00291581  9.9276e-5
 0.00163256   0.0215451   0.0891347     0.0470001  0.00599034  0.000239345
 0.00128422   0.0198886   0.0965585     0.0701159  0.0104871   0.000491717
 0.000860836  0.015645    0.0891347     0.0891347  0.015645    0.000860836
 0.000491717  0.0104871   0.0701159     0.0965585  0.0198886   0.00128422
 0.000239345  0.00599034  0.0470001  …  0.0891347  0.0215451   0.00163256
 9.9276e-5    0.00291581  0.0268469     0.0701159  0.0198886   0.00176853

In [ ]:
(πΩ * fill(1,7)')*ψϵω.([1], ϵ)

7×7 Matrix{Float64}:
 0.00637648  0.00790976  0.00895464  …  0.00895464  0.00790976  0.00637648
 0.0776814   0.0963606   0.10909        0.10909     0.0963606   0.0776814
 0.348144    0.431858    0.488907       0.488907    0.431858    0.348144
 0.573992    0.712014    0.806071       0.806071    0.712014    0.573992
 0.348144    0.431858    0.488907       0.488907    0.431858    0.348144
 0.0776814   0.0963606   0.10909     …  0.10909     0.0963606   0.0776814
 0.00637648  0.00790976  0.00895464     0.00895464  0.00790976  0.00637648

In [ ]:
# Chapter 3: System of Equations
const ΣW0 = W0

function φSYS(Ж, Ю, Б)
    XA = Ж[1:NS^2]
    XB = Ж[NS^2+1:2*NS^2]
    C1 = Ж[2*NS^2+1:3*NS^2]
    PX, PY = Ж[3*NS^2+1:3*NS^2+2]

    ΠAi = Б[1]
    ΠBi = Б[2]
    θXA = ΞθX[Б[3]]
    θXB = ΞθX[Б[4]]
    θY = ΞθY[Б[5]]

    #######sum this over 2D ##########

    # Conditional Expected Quantities
    SΩπ = φSΩπ(Ю)
    SΩ_XA_sum = sum(SΩπ .* XA', dims=2)
    SΩ_XB_sum = sum(SΩπ .* XB', dims=2)
    SΩ_C1_sum = sum(SΩπ .* C1', dims=2)

    # Aggregate Demand
    XΣ = SΩ_X_sum
    YΣ = (ΣW0 .- PX .* XΣ .- SΩ_C1_sum) ./ PY

    # Learning
    Ω = φΞΩI(Ю, XΣ, YΣ)

    # FOC's
    C2_values = (W0 .- PX .* X' .- C1') ./ PY .+ X' .* Δ2
    u_values = exp.(-γ .* C2_values)
    FOCX_values = β .* sum(Ω .* u_values .* (-PX ./ PY .+ Δ2), dims=1)
    FOCC1_values = exp.(-γ .* C1') + β .* sum(Ω .* u_values .* (-1.0 ./ PY), dims=1)

    # Market Clearing
    MCX = XΣ[ΞΠi[Πi]] - (θXbar - θX)
    MCY = YΣ[ΞΠi[Πi]] - (θYbar - θY)

    return vcat(FOCX_values', FOCC1_values', MCX, MCY)
end

φSYS (generic function with 1 method)

In [ ]:
# Chapter 4: Zero-Info Starting Point (ξ)
const W0ξ = W0

# Fixed Point Zero info
function φSYSξ(Жξ,Б)
    PXξ, PYξ = Жξ[1:2]
    θXA = ΞθX[Б[3]]
    θXB = ΞθX[Б[4]]
    θY = ΞθY[Б[5]]

    # Direct solution of MC's
    XAξ = (θXbar - θXA)
    XBξ = (θXbar - θXB)
    C1ξ = W0ξ - PXAξ * (θXbar - θXA) - PXBξ * (θXBbar - θXB) - PYξ *(θYbar - θY)
####
    # FOC's
    C2ξ_values = (W0ξ - PXAξ * XAξ - PXBξ * XBξ - C1ξ) / PYξ  .+ XAξ .* Δ2
    FOCXξ_values = β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-PXξ ./ PYξ .+ Δ2), dims=1)
    FOCC1ξ_values = exp.(-γ .* C1ξ) .+ β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-1.0 ./ PYξ), dims=1)

    return vcat(FOCXξ_values, FOCC1ξ_values)
end

# Starting Point of Full Code
function φЖ0(Б)
    θXi, θYi = Б[2], Б[3]
    θX = ΞθX[θXi]
    θY = ΞθY[θYi]
    solξ = nlsolve(Жξ -> φSYSξ(Жξ, [0, θXi, θYi]), [0.6, 0.7])
    PXξ, PYξ = solξ.zero
    Ж = zeros(2 * NS + 2)
    Ж[1:NS] .= θXbar - θX
    Ж[NS+1:2*NS] .= W0ξ .- PXξ * (θXbar .- θX) .- PYξ * (θYbar .- θY)
    Ж[2*NS+1:2*NS+2] .= [PXξ, PYξ]
    return Ж
end


φЖ0 (generic function with 1 method)

In [ ]:
# Chapter 5: Solution over grid

sol_zeros = Array{Array{Float64}, 3}(undef, NΠ, NθX, NθY)

for Πi in 1:NΠ
    for θXi in 1:NθX
        for θYi in 1:NθY
            Ж_ini = φЖ0([Πi, θXi, θYi])
            for Ю_value in range(0.05, stop=1, length=6)
                print("\rProcessing: Πi = $Πi, θXi = $θXi, θYi = $θYi, Ю_value = $Ю_value")
                flush(stdout)
                sol = nlsolve(Ж -> φSYS(Ж, [Ю_value], [Πi, θXi, θYi]), Ж_ini)
                Ж_ini = sol.zero
            end
            sol_zeros[Πi, θXi, θYi] = Ж_ini
        end
    end
end


Processing: Πi = 5, θXi = 5, θYi = 5, Ю_value = 1.0



---

